# Simulating test particle response to MHD

<img src='../../figures/mhd.png' alt='thumbnail' width='200'/>

It is possible to include rotating helical perturbations to simulations to e.g. study fast ion response to Alfvén eigenmodes and this tutorial shows how to do it.

We begin by generating a test case consisting of a 2D tokamak.

In [ ]:
import numpy as np
import unyt
import matplotlib.pyplot as plt
from a5py import Ascot

a5 = Ascot("ascot.h5", create=True)

# The magnetic input has to be B_2DS format so we use splines=True to convert
# the analytical field to splines
a5.data.create_input("bfield analytical iter circular", splines=True)
a5.data.create_input("wall_2D")
a5.data.create_input("plasma_1D")
a5.data.create_input("E_TC")
a5.data.create_input("N0_1D")
a5.data.create_input("asigma_loc")

print("Inputs created")

The MHD modes are defined in straight-field-line coordinates, which is why we need to construct mapping from cylindrical coordinates to Boozer coordinates.
While MHD can be included in all tokamak simulations, i.e. even those that have 3D field, the axisymmetric input is required to construct Boozer coordinates for the field and in a simulation the 3D field can be used.
This mapping is a separate input called ``boozer`` (it is user's responsibility to ensure ``bfield`` and ``boozer`` inputs are consistent), and there is a template to construct it automatically:


In [ ]:
a5.input_init(bfield=True)
a5.data.create_input("boozer tokamak", rhomin=0.05, rhomax=0.99)
a5.input_init(boozer=True) # Initialize also the Boozer data for plotting

We can plot the coordinates to make sure everything looks alright.
The defining feature of the Boozer coordinates is that the Jacobian, $J$, times the magnetic field squared, $JB^2$, is a flux quantity, so it is a good idea to check that as well.

In [ ]:
rgrid = np.linspace(4.3,8.3,100) * unyt.m
zgrid = np.linspace(-2,2,100) * unyt.m

fig = plt.figure()
ax1 = fig.add_subplot(2,2,1)
ax2 = fig.add_subplot(2,2,2)
ax3 = fig.add_subplot(2,2,3)
ax4 = fig.add_subplot(2,2,4)
a5.input_plotrz(rgrid, zgrid, "rho", axes=ax1)
a5.input_plotrz(rgrid, zgrid, "theta", axes=ax2)
# zeta changes from 0 to 2pi at phi=0 so we plot it at phi=180 instead
a5.input_plotrz(rgrid, zgrid, "zeta", axes=ax3, phi=180*unyt.deg)
a5.input_plotrz(rgrid, zgrid, "bjacxb2", axes=ax4)
plt.show()

Generating the Boozer coordinates near axis or separatrix may encounter issues, which is why it is a good idea to use the limits ``rhomin`` and ``rhomax`` to control what area the coordinates cover.
Outside this area the MHD input is not evaluated so this it only needs to cover the region where the modes are active, and limiting the region decreases the CPU time needed to run the simulation.

Now let's plot the $q$-profile before generating the MHD input.

In [ ]:
rho = np.linspace(0,1,100)
q, I, g = a5.input_eval_safetyfactor(rho)
plt.plot(rho, q)

Notice that the q-profile is ill-defined close to the axis, which is caused by the same issue that prevents creation of the Boozer coordinates at that point.

There is rational $q=2$ surface around $\rho=0.8$, which is where we initialize our MHD mode.
Multiple modes can be included in a simulation and they can have time-dependent eigenmodes (though those increase CPU cost considerably).
However, for this tutorial we initialize justa a single $(n=1,m=2)$ mode that peaks at the rational surface.

In [ ]:
mhd = {
    "nmode" : 1, # Number of modes
    "nmodes" : np.array([1]), "mmodes" : np.array([2]), # Mode tor and pol numbers
    "amplitude" : np.array([1.0]), "omega" : np.array([50.0e3]), "phase" : np.array([0.0]),
    "nrho" : 200, "rhomin" : 0.0, "rhomax" : 1.0
   }

# Eigenmodes are given in the usual sqrt of normalized poloidal flux grid
rhogrid = np.linspace(mhd["rhomin"], mhd["rhomax"], mhd["nrho"])
alpha   = np.exp( -(rhogrid-0.8)**2/0.005 ) # Magnetic potential
phi     = alpha*0 # Electric perturbation potential, we will come back to this

mhd["phi"]   = np.tile(phi,   (mhd["nmode"],1)).T
mhd["alpha"] = np.tile(alpha, (mhd["nmode"],1)).T
a5.data.create_input("MHD_STAT", **mhd, desc="UNSCALED")

plt.figure()
plt.plot(rhogrid, alpha)
plt.show()

We used the tag "UNSCALED" for this input to notify that it is not suitable for a simulation yet.
When using data provided by other codes, the MHD input is usually unscaled meaning that the eigenmodes are otherwise fine, but they have to be scaled by the ``amplitude`` parameter so that we get the desired perturbation level $\delta B/B$.

So now let's initialize the MHD input and plot the perturbation level.

In [ ]:
# Note that plotting MHD requires that both bfield and boozer are also initialized
# but those we have initialized earlier in this tutorial.
a5.input_init(mhd=a5.data.mhd.UNSCALED.get_qid())
a5.input_plotrz(rgrid, zgrid, "db/b (mhd)")
a5.input_free(mhd=True)

For this tutorial, we desire something like $\delta B/B \approx 10^{-3}$.
So let's read the input and fix the amplitude.

In [ ]:
mhd = a5.data.mhd.UNSCALED.read()
mhd["amplitude"][:] = 1e-3 / 8.1

a5.data.create_input("MHD_STAT", **mhd, desc="SCALED")
a5.input_init(mhd=a5.data.mhd.SCALED.get_qid())
a5.input_plotrz(rgrid, zgrid, "db/b (mhd)")